In [3]:
import yfinance as yf
import pmdarima
import plotly
import plotly.graph_objs as go

# getting data from Yahoo Finance
stock_name = 'AMD'  # here you can change the name of stock ticker, for example we will take AMD ticker
data = yf.download(stock_name, start="2023-01-01", end="2024-01-30")
data_adf = data.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1)
data_adf = data_adf['Close']


from pmdarima.arima import ADFTest
adf_test = ADFTest(alpha = 0.05)
adf_test.should_diff(data_adf)

[*********************100%%**********************]  1 of 1 completed


(0.9459530446188531, True)

In [4]:
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pmdarima as pm
plt.style.use('fivethirtyeight')
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import ADFTest
from pmdarima.datasets import load_wineind
import random

def arima(stock_name, data):
  df_close = data['Close']

    # Split data into train and test set (90% - train, 10% - test)
  df_log = df_close
    #train_data, test_data = df_log[3:int(len(df_log) * 0.9)], df_log[int(len(df_log) * 0.9):]
  train_data, test_data = df_log[3:int(len(df_log) * 0.9)], df_log[int(len(df_log) * 0.9):]
  test_values = len(df_log) * 0.01 + 1.0
  x_train = list(range(0, 224))
  x_test = list(range(224, int(len(data))))
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=x_train, y=train_data, mode='lines+markers', marker=dict(size=4),  name='train', marker_color='#39304A'))
  fig.add_trace(go.Scatter(x=x_test, y=test_data, mode='lines+markers', marker=dict(size=4), name='test', marker_color='#A98D75'))
  fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  plot_bgcolor='#FFFFFF',
                  xaxis=dict(gridcolor = 'lightgrey'),
                  yaxis=dict(gridcolor = 'lightgrey'),
                  title_text = f'{stock_name} ARIMA data', title_x = 0.5,
                  xaxis_title="Timestep",
                  yaxis_title="Stock price",
                  margin=dict(l=0, r=0, t=30, b=0))
  fig.show()
  model =  pm.auto_arima(df_log,start_p=0, d=None, start_q=0,
                          max_p=5, max_d=5, max_q=5, start_P=0,
                          D=1, start_Q=0, max_P=5, max_D=5,
                          max_Q=5, m=7, seasonal=True,
                          error_action='warn',trace = True,
                          supress_warnings=True,stepwise = True,
                          random_state=20,n_fits = 50 )

  model.summary()
  exo_data = data['Volume']
  exo_data = exo_data[int(len(exo_data) * 0.9):]

  preds = model.predict(n_periods = 22, X = exo_data)

  preds = np.vstack(preds)
  hist_data = yf.download(stock_name, start="2021-04-01", end="2021-05-04")
  hist_data = hist_data.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1)
  hist_data = hist_data['Close']
  hist_data = np.array(hist_data)

  rmse = np.sqrt(np.mean(((preds - hist_data) ** 2)))
  print(f'RMSE ARIMA: {rmse}')
  preds_gr = np.reshape(preds, (22,))
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=list(range(0, 21)), y=hist_data, mode='lines+markers',  name='historical', marker_color='#39304A'))
  fig.add_trace(go.Scatter(x=list(range(0, 21)), y=preds_gr, mode='lines+markers', name='predictions', marker_color='#FFAA00'))
  fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  plot_bgcolor='#FFFFFF',
                  xaxis=dict(gridcolor = 'lightgrey'),
                  yaxis=dict(gridcolor = 'lightgrey'),
                  title_text = f'{stock_name} ARIMA prediction', title_x = 0.5,
                  xaxis_title="Timestep",
                  yaxis_title="Stock price",
                  margin=dict(l=0, r=0, t=30, b=0))
  fig.show()

  return preds, rmse


In [5]:
arima(stock_name, data)

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,0)[7] intercept   : AIC=1851.445, Time=0.13 sec
 ARIMA(1,0,0)(1,1,0)[7] intercept   : AIC=1442.376, Time=0.71 sec
 ARIMA(0,0,1)(0,1,1)[7] intercept   : AIC=1671.253, Time=0.78 sec
 ARIMA(0,0,0)(0,1,0)[7]             : AIC=1880.586, Time=0.10 sec
 ARIMA(1,0,0)(0,1,0)[7] intercept   : AIC=1492.401, Time=0.15 sec
 ARIMA(1,0,0)(2,1,0)[7] intercept   : AIC=1412.662, Time=1.33 sec
 ARIMA(1,0,0)(3,1,0)[7] intercept   : AIC=1407.229, Time=3.12 sec
 ARIMA(1,0,0)(4,1,0)[7] intercept   : AIC=1404.176, Time=7.65 sec
 ARIMA(1,0,0)(5,1,0)[7] intercept   : AIC=1405.314, Time=11.78 sec
 ARIMA(1,0,0)(4,1,1)[7] intercept   : AIC=inf, Time=6.69 sec
 ARIMA(1,0,0)(3,1,1)[7] intercept   : AIC=inf, Time=5.58 sec
 ARIMA(1,0,0)(5,1,1)[7] intercept   : AIC=inf, Time=12.74 sec
 ARIMA(0,0,0)(4,1,0)[7] intercept   : AIC=1847.169, Time=2.40 sec
 ARIMA(2,0,0)(4,1,0)[7] intercept   : AIC=1406.168, Time=3.56 sec
 ARIMA(1,0,1)(4,1,0)[7] intercept   : AIC=1406

[*********************100%%**********************]  1 of 1 completed

 ARIMA(1,0,0)(4,1,0)[7]             : AIC=1404.992, Time=1.20 sec

Best model:  ARIMA(1,0,0)(4,1,0)[7] intercept
Total fit time: 72.653 seconds
RMSE ARIMA: 104.519185497432


(array([[183.09693507],
        [180.24702562],
        [177.27861171],
        [179.24779281],
        [181.24768479],
        [180.98507222],
        [183.30895996],
        [187.11581517],
        [184.43904508],
        [180.8888539 ],
        [182.31466106],
        [185.78320935],
        [185.93372216],
        [188.23752684],
        [193.46585413],
        [189.85940344],
        [186.48039788],
        [188.56039912],
        [191.7382963 ],
        [191.1137098 ],
        [193.41914932],
        [198.56474833]]),
 104.519185497432)